In [6]:
import arkouda as ak
import arachne as ar
import scipy as sp
import networkx as nx
import matplotlib.pyplot as plt
import os
import time

ak.connect("n28", 5555)
p = 0.5
num_nodes = 10

seed = 42

# Define probabilities for labels and relationships
node_lbl_probs = {"lbls2": [1, 0.0],  # Probabilities for integers 10 and 11
                  "lbls3": [1, 0.0]}  # Probabilities for True and False
edge_rel_probs = {"rels1": [1, 0.0],  # Probabilities for integers 5 and 10
                  "rels2": [1, 0.0]}  # Probabilities for True and False


start = time.time()
temp_prop_graph = ar.gnp(num_nodes, p, create_using=ar.PropGraph, seed=seed)
end = time.time()
build_time = end - start

print(f"Building property graph with {len(temp_prop_graph)} vertices and "
      f"{temp_prop_graph.size()} "
      f"edges took {round(build_time,2)} seconds.")

### Generate node labels and edge relationships for the graph.
# 1. Extract node and edge information.
num_edges = temp_prop_graph.size()
num_nodes = len(temp_prop_graph)
edges = temp_prop_graph.edges()
nodes = temp_prop_graph.nodes()

##############################################
# Generate random node and edge attributes for the main graph
##############################################

# # For nodes:
# node_ints = ak.randint(10, 12, num_nodes, seed=seed)
# node_bools = ak.randint(0, 2, num_nodes, dtype=ak.bool, seed=seed)

# # For edges:
# edge_ints = ak.randint(10, 12, num_edges, seed=seed)

# For nodes
node_ints = ak.where(ak.randint(0, 100, num_nodes, seed=seed) < node_lbl_probs["lbls2"][0] * 100, 10, 11)
node_bools = ak.randint(0, 100, num_nodes, seed=seed) < node_lbl_probs["lbls3"][0] * 100

# For edges
edge_ints = ak.where(ak.randint(0, 100, num_edges, seed=seed) < edge_rel_probs["rels1"][0] * 100, 5, 10)
edge_bools = ak.randint(0, 100, num_edges, seed=seed) < edge_rel_probs["rels2"][0] * 100


# Create dataframes with the new attributes
edge_df = ak.DataFrame({
    "src": edges[0],
    "dst": edges[1],
    "rels1": edge_ints,
    "rels2": edge_bools
})

node_df = ak.DataFrame({
    "nodes": nodes,
    "lbls2": node_ints,
    "lbls3": node_bools
})

# Create the new property graph with these attributes
prop_graph = ar.PropGraph()
prop_graph.load_edge_attributes(edge_df, source_column="src", destination_column="dst")
prop_graph.load_node_attributes(node_df, node_column="nodes")
print("Property graph created with random node and edge attributes.")

##############################################
# Create the subgraph and assign random attributes
##############################################

# Subgraph structure
src_list = [2, 3, 1, 3]
dst_list = [3, 1, 2, 0]
src_subgraph = ak.array(src_list)
dst_subgraph = ak.array(dst_list)

subgraph_nodes = list(set(src_list) | set(dst_list))
subgraph_nodes.sort()

# Generate random node and edge attributes for the subgraph
num_subgraph_nodes = len(subgraph_nodes)
num_subgraph_edges = len(src_list)

# subgraph_node_ints = ak.randint(10, 12, num_subgraph_nodes, seed=seed)
# subgraph_node_bools = ak.randint(0, 2, num_subgraph_nodes, dtype=ak.bool, seed=seed)
# subgraph_edge_ints = ak.randint(10, 12, num_subgraph_edges, seed=seed)

# # Subgraph attributes
# subgraph_node_ints = ak.where(ak.randint(0, 100, num_subgraph_nodes, seed=seed) < node_lbl_probs["lbls2"][0] * 100, 10, 11)
# subgraph_node_bools = ak.randint(0, 100, num_subgraph_nodes, seed=seed) < node_lbl_probs["lbls3"][0] * 100

# subgraph_edge_ints = ak.where(ak.randint(0, 100, num_subgraph_edges, seed=seed) < edge_rel_probs["rels1"][0] * 100, 5, 10)
# subgraph_edge_bools = ak.randint(0, 100, num_subgraph_edges, seed=seed) < edge_rel_probs["rels2"][0] * 100

# Fixed attributes
subgraph_node_ints = ak.array([10, 10, 10, 10])
subgraph_node_bools = ak.array([True, True, True, True])
subgraph_edge_ints = ak.array([5, 5, 5, 5])
subgraph_edge_bools = ak.array([True, True, True, True])

# Create dataframes for subgraph attributes
edge_df_h = ak.DataFrame({
    "src": src_subgraph,
    "dst": dst_subgraph,
    "rels1": subgraph_edge_ints,
    "rels2": subgraph_edge_bools
})

node_df_h = ak.DataFrame({
    "nodes": ak.array(subgraph_nodes),
    "lbls2": subgraph_node_ints,
    "lbls3": subgraph_node_bools
})

# Create the subgraph with these attributes
sg = ar.PropGraph()
sg.load_edge_attributes(edge_df_h, source_column="src", destination_column="dst")
sg.load_node_attributes(node_df_h, node_column="nodes")



print("Subgraph created with random node and edge attributes.")


connected to arkouda server tcp://*:5555


Building property graph with 10 vertices and 49 edges took 2.2 seconds.
Property graph created with random node and edge attributes.
Subgraph created with random node and edge attributes.


In [ ]:
import arkouda as ak
import arachne as ar
import numpy as np

# Perform subgraph isomorphism search
isos_as_vertices = ar.subgraph_isomorphism(
    prop_graph, sg,
    semantic_check="and", algorithm_type="ps",
    reorder_type=None, return_isos_as="vertices"
)

# Calculate the number of isomorphisms found
num_isos = len(isos_as_vertices[0]) // len(src_subgraph)
print(f"We found {num_isos} monos inside of the graph")

if num_isos == 0:
    print("No isomorphisms found. Exiting.")
else:
    # Pick 1% of isomorphisms randomly
    num_to_modify = max(1, int(0.01 * num_isos))  # Ensure at least 1 isomorphism is modified
    selected_indices = np.random.choice(range(num_isos), size=num_to_modify, replace=False)

    print(f"Selected isomorphism indices: {selected_indices}")

    # Ensure subgraph_nodes is converted to an Arkouda pdarray
    if isinstance(subgraph_nodes, list):
        subgraph_nodes = ak.array(subgraph_nodes)

    # Convert Arkouda pdarrays to numpy arrays for processing
    subgraph_nodes_nd = subgraph_nodes.to_ndarray()
    isos_as_vertices_nd = isos_as_vertices[0].to_ndarray()

    selected_edge_indices = []

    for iso_idx in selected_indices:
        # Get the mapping for this isomorphism
        start_idx = iso_idx * len(src_subgraph)
        end_idx = start_idx + len(src_subgraph)
        subgraph_to_graph_mapping = dict(zip(subgraph_nodes_nd, isos_as_vertices_nd[start_idx:end_idx]))

        print(f"Subgraph-to-graph mapping for isomorphism {iso_idx}: {subgraph_to_graph_mapping}")

        # For each edge in the subgraph, find its corresponding edge in the main graph
        for src, dst in zip(src_subgraph, dst_subgraph):
            mapped_src = subgraph_to_graph_mapping[src]
            mapped_dst = subgraph_to_graph_mapping[dst]

            # Find the index of this edge in the main graph
            edge_index = ((edges[0] == mapped_src) & (edges[1] == mapped_dst)).nonzero()[0]
            if len(edge_index) > 0:
                selected_edge_indices.append(edge_index[0])

    # Update node attributes for selected nodes
    selected_node_indices = ak.concatenate([isos_as_vertices[0][iso_idx * len(src_subgraph):(iso_idx + 1) * len(src_subgraph)] for iso_idx in selected_indices])
    print(f"All selected node indices: {selected_node_indices}")

    print("Node DataFrame before update:")
    print(node_df)

    node_df["lbls2"][selected_node_indices] = 10  # Update lbls2 to 10
    node_df["lbls3"][selected_node_indices] = True  # Update lbls3 to True

    print("Node DataFrame after update:")
    print(node_df)

    # Update edge attributes for the selected edges
    selected_edge_indices = ak.array(selected_edge_indices)
    print(f"All selected edge indices: {selected_edge_indices}")

    print("Edge DataFrame before update:")
    print(edge_df)

    edge_df["rels1"][selected_edge_indices] = 5  # Update rels1 to 5
    edge_df["rels2"][selected_edge_indices] = True  # Update rels2 to True

    print("Edge DataFrame after update:")
    print(edge_df)

    # Reload the updated attributes into the graph
    prop_graph.load_node_attributes(node_df, node_column="nodes")
    prop_graph.load_edge_attributes(edge_df, source_column="src", destination_column="dst")

    print(f"Updated attributes for {num_to_modify} randomly selected isomorphisms.")


In [ ]:
"""VF2-SI """
isos_as_vertices = ar.subgraph_isomorphism(prop_graph, sg, 
                                           semantic_check = "and", algorithm_type = "si",
                                           reorder_type = "structural", return_isos_as = "vertices")

print(f"We found {len(isos_as_vertices[0])/len(sg)} monos inside of the graph")

In [ ]:
"""VF2-SI PROBABILITY-MVE"""
isos_as_vertices = ar.subgraph_isomorphism(prop_graph, sg, 
                                           semantic_check = "and", algorithm_type = "si",
                                           reorder_type = "probability", return_isos_as = "vertices")

print(f"We found {len(isos_as_vertices[0])/len(sg)} monos inside of the graph")

In [7]:
"""VF2-PS DEFAULT"""
isos_as_vertices = ar.subgraph_isomorphism(prop_graph, sg, 
                                           semantic_check = "and", algorithm_type = "ps", 
                                           reorder_type = None, return_isos_as = "vertices")

print(f"We found {len(isos_as_vertices[0])/len(sg)} monos inside of the graph")
#print(isos_as_vertices)

We found 407.0 monos inside of the graph


In [ ]:
"""VF2-PS MVE-REORDERING"""
isos_as_vertices = ar.subgraph_isomorphism(prop_graph, sg, 
                                           semantic_check = "and", algorithm_type = "ps", 
                                           reorder_type = "structural", return_isos_as = "vertices")

print(f"We found {len(isos_as_vertices[0])/len(sg)} monos inside of the graph")

In [ ]:
"""VF2-PS PROBABILITY-MVE"""
isos_as_vertices = ar.subgraph_isomorphism(prop_graph, sg, 
                                           semantic_check = "and", algorithm_type = "ps", 
                                           reorder_type = "probability", return_isos_as = "vertices")

print(f"We found {len(isos_as_vertices[0])/len(sg)} monos inside of the graph")

In [ ]:
"""Correctness check with NetworkX """
# Get node and edge attributes from Arachne property graphs.
subgraph_node_attributes = sg.get_node_attributes()
subgraph_edge_attributes = sg.get_edge_attributes()
graph_node_attributes = prop_graph.get_node_attributes()
graph_edge_attributes = prop_graph.get_edge_attributes()

# Create NetworkX subgraph.
subgraph_networkx = nx.from_pandas_edgelist(subgraph_edge_attributes.to_pandas(), source="src", target="dst", edge_attr=True, create_using=nx.DiGraph)
subgraph_node_attribute_dict = subgraph_node_attributes.to_pandas().set_index('nodes').to_dict('index')
print(subgraph_node_attribute_dict)
nx.set_node_attributes(subgraph_networkx, subgraph_node_attribute_dict)

# Create NetworkX main graph.
graph_networkx = nx.from_pandas_edgelist(graph_edge_attributes.to_pandas(), source="src", target="dst", edge_attr=True, create_using=nx.DiGraph)
graph_node_attribute_dict = graph_node_attributes.to_pandas().set_index('nodes').to_dict('index')
print(graph_node_attribute_dict)
nx.set_node_attributes(graph_networkx, graph_node_attribute_dict)

# Attribute matching functions that need to be used by the NetworkX DiGraphMatcher.
def node_matcher(u, v):
    return u == v

def edge_matcher(e1, e2):
    return e1 == e2

# Find subgraph isomorphisms of H in G.
structural_matcher = nx.algorithms.isomorphism.DiGraphMatcher(graph_networkx, subgraph_networkx)
attribute_matcher = nx.algorithms.isomorphism.DiGraphMatcher(graph_networkx, subgraph_networkx, node_match=node_matcher, edge_match=edge_matcher)

# List of dicts. For each dict, keys is original graph vertex, values are subgraph vertices.
subgraph_isomorphisms_structural = list(structural_matcher.subgraph_monomorphisms_iter())
print("Structural monomorphisms found =", len(subgraph_isomorphisms_structural))

start_time = time.time()

subgraph_isomorphisms_attributed = list(attribute_matcher.subgraph_monomorphisms_iter())
end_time = time.time()

elapsed_time = end_time - start_time
print("Attributed monomorphisms found =", len(subgraph_isomorphisms_attributed))
print(f"Time taken to find attributed monomorphisms: {elapsed_time:.2f} seconds")


In [ ]:
"""VF2-SI No Label No Rel"""
edge_df_h1 = ak.DataFrame({"src": src_subgraph, "dst": dst_subgraph})

sg_1 = ar.PropGraph()
sg_1.load_edge_attributes(edge_df_h1, source_column="src", destination_column="dst")

isos_as_vertices = ar.subgraph_isomorphism(prop_graph, sg_1, 
                                           semantic_check = "and", algorithm_type = "si",
                                           reorder_type = "structural", return_isos_as = "vertices")

print(f"We found {len(isos_as_vertices[0])/len(sg_1)} monos inside of the graph")

In [ ]:
"""VF2-PS MVE-REORDERING No Label No Rel"""
edge_df_h1 = ak.DataFrame({"src": src_subgraph, "dst": dst_subgraph})

sg_1 = ar.PropGraph()
sg_1.load_edge_attributes(edge_df_h1, source_column="src", destination_column="dst")

isos_as_vertices = ar.subgraph_isomorphism(prop_graph, sg, 
                                           semantic_check = "and", algorithm_type = "ps", 
                                           reorder_type = "structural", return_isos_as = "vertices")

print(f"We found {len(isos_as_vertices[0])/len(sg)} monos inside of the graph")